**Import necessary package**

In [4]:
import numpy as np
import matplotlib.pyplot as plt
#from scipy.misc import imread
from IPython import display
from PIL import Image
%matplotlib inline

In [5]:
import os
import cv2
from tqdm import tqdm

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

**Data Pre-Processing**

In [ ]:
# 

**NN Model**

In [26]:
class Net(nn.Module):
    
    def __init__(self, imput_size, hidden_size1, hidden_size2, hidden_size3, kernel_size, output_size):
        super().__init__()
        self.conv1 = nn.Conv2d(imput_size, hidden_size1, kernel_size) 
        self.conv2 = nn.Conv2d(hidden_size1, hidden_size2, kernel_size) 
        self.conv3 = nn.Conv2d(hidden_size2, hidden_size3, kernel_size) 
                
        x = torch.randn(50, 50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
        
        # Eventually output to at least one linear layer
        # self.fc1 = nn.Linear(???, 512)
        self.fc1 = nn.Linear(self._to_linear, 64)
        self.fc2 = nn.Linear(64, output_size)        
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        #print(x[0].shape)        
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
            #print(self._to_linear)
        return x
    
    def forward(self, x):
        x = self.convs(x)
        xF = x.view(-1, self._to_linear) # flatten
        output = F.relu(self.fc1(xF)) # put into the first fully connected layer
        output = self.fc2(x)
        return F.softmax(output, dim=1)

net = Net(1, 32, 64, 128, 5, 5)